In [1]:
import snap
S = 10**4
TPATH = "/courses/TSKS33/ht2023/data/"
def genmod10star():   
    #G1 = snap.GenPrefAttach(1000, 10)
    #G1 = snap.GenRndPowerLaw(100000,2.2)
    G = snap.GenStar(snap.PUNGraph,10,False)
    G.AddEdge(4,5)
    #G = snap.GetMxScc(G1)
    N = G.GetNodes()
    
    # node numbering in the mat file is sequential n=1,2,... following the node iterator
    # gio.h line 369
#    snap.SaveMatlabSparseMtx(G, "mod10star.mat")

    # assign the degree as attribute
    x = snap.TIntFltH(N)
#    file = open("mod10star_attr.mat",'w') 
    for NI in G.Nodes():
        n=NI.GetId()
        x[n]=NI.GetDeg()
#        file.write(str(n+1)+"\t"+str(x[n])+"\n")
#        file.write(str(x[n])+"\n")
#    file.close()
    
    return G, x


In [2]:
G, x = genmod10star()

sum = 0
for n in G.Nodes():
    sum+=n.GetDeg()
exact = sum/G.GetNodes()
print("-- expected values of <x>-hat -----")
print("uniform sampling:", exact)

total_expected_degree = 0
for i in G.Nodes():
    node = G.GetNI(i.GetId())
    tmp = 0
    for nbr_id in node.GetOutEdges():
        nbr = G.GetNI(nbr_id)
        tmp += nbr.GetDeg()
    total_expected_degree += (tmp / node.GetDeg())

# Print the result
print("random connection of random node:", round(total_expected_degree/G.GetNodes(), 3))

-- expected values of <x>-hat -----
uniform sampling: 2.0
random connection of random node: 7.522


In [37]:
total_expected_degree = 0
sum = 0
for i in G.Nodes():
    node = G.GetNI(i.GetId())
    sum += node.GetDeg()/G.Get

print(sum)

2.0000000000000004


In [ ]:
num_runs = 5
for i in range(num_runs):
    tmp_sum = 0
    Rnd = snap.TRnd(42)
    Rnd.Randomize()
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
        tmp_sum += node.GetDeg()
    print("uniform sampling:", tmp_sum/S)


uniform sampling: 2.0173
uniform sampling: 2.0029
uniform sampling: 2.0557
uniform sampling: 1.9839
uniform sampling: 1.969


In [ ]:
import random
for i in range(num_runs):
    sum = 0
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        nbr = G.GetNI(nbr)
        sum+=nbr.GetDeg()

    print("random connection of random node:", sum/S)

random connection of random node: 7.542
random connection of random node: 7.5532
random connection of random node: 7.4758
random connection of random node: 7.5411
random connection of random node: 7.4898


In [ ]:
for i in range(num_runs):
    sum = 0
    node = G.GetNI(0)
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
    for i in range(S):
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        node = G.GetNI(nbr)
        sum += node.GetDeg()
        

    print("uniform random walk:", sum/S)

uniform random walk: 4.7935
uniform random walk: 4.7983
uniform random walk: 4.8075
uniform random walk: 4.8118
uniform random walk: 4.7749


In [ ]:
for i in range(num_runs):
    sum = 0
    node = G.GetNI(0)
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)

    for i in range(S):
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        nbr = G.GetNI(nbr)
        nbr_deg = nbr.GetDeg()
        node_deg = node.GetDeg()
        sum += node_deg
        if random.uniform(0, 1) < (node_deg/nbr_deg):
            node = nbr
    print("uniform random walk:", sum/S)

uniform random walk: 1.9805
uniform random walk: 1.9974
uniform random walk: 1.9937
uniform random walk: 1.9433
uniform random walk: 1.977
